### **Cause Prediction Inference**

In [1]:
# !pip install -r "requirements.txt"

In [2]:
from accelerate import Accelerator
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
)
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

D:\MiniConda\envs\pythonProject6\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with_labels = False

In [4]:
def get_prompt_without_output(sample_data_point, speaker=False, video=False, audio=False):
	if speaker and video and audio:
		instruction = """Identify the cause utterances of the emotion in utterance {}. The video_captions for each utterance describe the video corresponding to the text.\nOutput in one line the ids of the cause utterances as a list in the given format:\nutterance_ids :: [cause utterance ids]\nDon't give any explanation.""".format(sample_data_point["utterance_id"])
	elif speaker and video:
		instruction = """Identify the cause utterances of the emotion in utterance {}. The video_captions for each utterance describe the video corresponding to the text.\nOutput in one line the ids of the cause utterances as a list in the given format:\nutterance_ids :: [cause utterance ids]\nDon't give any explanation.""".format(sample_data_point["utterance_id"])
	else:
		instruction = """Identify the cause utterances of the emotion in utterance {}. Output in one line the ids of the cause utterances as a list in the given format:\nutterance_ids :: [cause utterance ids]\nDon't give any explanation.""".format(sample_data_point["utterance_id"])

	return f"""<s>[INST]\n{sample_data_point['conversation']}{instruction}\n[/INST]\n"""

In [5]:
def get_prompt_with_output(sample_data_point, speaker=False, video=False, audio=False):
	if speaker and video and audio:
		instruction = """Identify the cause utterances of the emotion in utterance {}. The video_captions for each utterance describe the video corresponding to the text.\nOutput in one line the ids of the cause utterances as a list in the given format:\nutterance_ids :: [cause utterance ids]\nDon't give any explanation.""".format(sample_data_point["utterance_id"])
	elif speaker and video:
		instruction = """Identify the cause utterances of the emotion in utterance {}. The video_captions for each utterance describe the video corresponding to the text.\nOutput in one line the ids of the cause utterances as a list in the given format:\nutterance_ids :: [cause utterance ids]\nDon't give any explanation.""".format(sample_data_point["utterance_id"])
	else:
		instruction = """Identify the cause utterances of the emotion in utterance {}. Output in one line the ids of the cause utterances as a list in the given format:\nutterance_ids :: [cause utterance ids]\nDon't give any explanation.""".format(sample_data_point["utterance_id"])

	return f"""<s>[INST]\n{sample_data_point['conversation']}{instruction}\n[/INST]\n""", f"""utterance_ids :: {sample_data_point['causes_label']}\n</s>"""

In [6]:
def get_formatted_conversation(data_sample, speaker=False, video=False, audio=False):
    conversation_str = '"conversation": [\n'
    for utt in data_sample["conversation"]:
        conversation_str += "{{\n\"utterance_ID\": {}\n\"text\": {}\n\"speaker\": {}\n\"emotion\": {}\n}}\n".format(utt["utterance_ID"], utt["text"], utt["speaker"], utt["emotion"])
    return conversation_str

In [7]:
def get_causes_list_str(cause_list):
    str_out = "["
    i = 0
    while i < len(cause_list):
        if i == len(cause_list)-1:
            str_out += str(cause_list[i])
        else:
            str_out += str(cause_list[i]) + ","
        i += 1
    str_out += "]"
    return str_out

In [8]:
def form_input(data_sample, utt_id, speaker=False, video=False, audio=False):
    conversation_str = get_formatted_conversation(data_sample, speaker, video, audio)
    emotions_str = data_sample["conversation"][utt_id]["emotion"]
    if with_labels:
        causes_list_str = get_causes_list_str(data_sample["conversation"][utt_id]["cause_list"])

    sample_data_point = {} # has utterance_id, conversation, emotion_label, conv id

    sample_data_point["utterance_id"] = utt_id+1
    sample_data_point["conversation"] = conversation_str
    sample_data_point["emotion_label"] = emotions_str
    if with_labels:
        sample_data_point["causes_label"] = causes_list_str
    sample_data_point["causes_list"] = data_sample["conversation"][utt_id]["cause_list"]
    sample_data_point["conversation_id"] = data_sample["conversation_ID"]
    return sample_data_point

In [9]:
def generate_inst_for_each_utt_of_conv(data_point, speaker=False, video=False, audio=False):
    num_utt = len(data_point["conversation"])
    list_output_objects = []
    for i in range(num_utt):
        sample_data_point = form_input(data_point, i, speaker, video, audio)
        if with_labels:
            inst, out = get_prompt_with_output(sample_data_point, speaker, video, audio)
        else:
            inst = get_prompt_without_output(sample_data_point, speaker, video, audio)
        if with_labels:
            obj = {
                "instruction": inst,
                "out": out,
                "text": inst + out,
                "emotion_label": sample_data_point["emotion_label"],
                "utterance_id": sample_data_point["utterance_id"],
                "conversation_id": sample_data_point["conversation_id"],
                "causes_label":sample_data_point["causes_label"],
                "causes_list":sample_data_point["causes_list"]
            }
        else:
            obj = {
                "instruction": inst,
                "emotion_label": sample_data_point["emotion_label"],
                "utterance_id": sample_data_point["utterance_id"],
                "conversation_id": sample_data_point["conversation_id"],
            }
        list_output_objects.append(obj)
    return list_output_objects

In [10]:
def process_dataset(data, speaker=False, video=False, audio=False):
    flattened_list = [element for sublist in map(lambda x: generate_inst_for_each_utt_of_conv(x, speaker, video, audio), data) for element in sublist]
    ds = {}
    # IMP: only include those utterances in dataset where emotion is not "neutral", we will only find cause list for those
    ds["instruction"] = [sample["instruction"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
    ds["emotion_label"] = [sample["emotion_label"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
    ds["utterance_id"] = [sample["utterance_id"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
    ds["conversation_id"] = [sample["conversation_id"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
    if with_labels:
        ds["causes_label"] = [sample["causes_label"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
        ds["causes_list"] = [sample["causes_list"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
        ds["out"] = [sample["out"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
        ds["text"] = [sample["text"] for sample in flattened_list if sample["emotion_label"] != "neutral"]
    return ds

In [15]:
from sklearn.model_selection import train_test_split

def create_json(data, save_folder, speaker=False, video=False, audio=False):

    if not os.path.exists(save_folder):
        os.makedirs(save_folder, exist_ok=True)

    if with_labels:
        # create a map for storing cause_list for each utt of each conv
        # map of map
        cause_map = {}
        for conv in data:
            conv_id = conv["conversation_ID"]
            cause_map[conv_id] = {}
            for pair in conv["emotion-cause_pairs"]:
                emo_id = int(pair[0].split('_')[0])
                cau_id = int(pair[1])
                if emo_id in cause_map[conv_id].keys():
                    cause_map[conv_id][emo_id].append(cau_id)
                else:
                    cause_map[conv_id][emo_id] = [cau_id]
        # Now go through utt and add their "cause_list", could be [] for neutral
        for conv in data:
            conv_id = conv["conversation_ID"]
            if conv_id not in cause_map.keys():
                for utt in conv["conversation"]:
                    utt["cause_list"] = []
            else:
                for utt in conv["conversation"]:
                    utt_id = utt["utterance_ID"]
                    if utt_id not in cause_map[conv_id].keys():
                        utt["cause_list"] = []
                    else:
                        utt["cause_list"] = cause_map[conv_id][utt_id]
    else:
        for conv in data:
            for utt in conv["conversation"]:
                utt["cause_list"] = []


    print("Number of conversations in dataset: {}".format(len(data)))

    conv_file = save_folder + "conversations.json"
    with open(conv_file, 'w') as f:
        json.dump(data, f)
    return conv_file

def generate_input(args):
    text_folder = "./results_test/"
    save_dir = "cause_input"
    
    file_path = text_folder + "emotion_labelled_data.json"
    file = open(file_path)
    data = json.load(file)

    test_file = create_json(data, save_dir, args.speaker, args.video, args.audio)
    return test_file

In [16]:
# Params
class Config:
    def __init__(self) -> None:
        self.seed = 42
        self.speaker = True
        self.video = False
        self.audio = False

In [17]:
data = []

In [18]:
config = Config()
from datasets import Dataset

if not with_labels:
    data_file = generate_input(config)
    with open(data_file, 'r') as f:
        data = json.load(f)
        dataset = process_dataset(data, speaker=config.speaker, video=config.video)
        dataset = Dataset.from_dict(dataset)
        print("Number of utterances {}".format(len(dataset["instruction"])))


Number of conversations in dataset: 665
Number of utterances 2274


In [19]:
# !huggingface-cli login

In [20]:
model_id = "models--Undi95--Meta-Llama-3-8B-Instruct-hf"

In [21]:
import accelerate
model_config = AutoConfig.from_pretrained(model_id)

with accelerate.init_empty_weights():
    fake_model = AutoModelForCausalLM.from_config(model_config)

device_map=accelerate.infer_auto_device_map(fake_model, max_memory={0: "24GiB", "cpu": "20GiB"}) #
print(json.dumps(device_map, indent=4))

{
    "model.embed_tokens": 0,
    "model.layers.0": 0,
    "model.layers.1": 0,
    "model.layers.2": 0,
    "model.layers.3": 0,
    "model.layers.4": 0,
    "model.layers.5": 0,
    "model.layers.6": 0,
    "model.layers.7": 0,
    "model.layers.8": 0,
    "model.layers.9": 0,
    "model.layers.10": 0,
    "model.layers.11": 0,
    "model.layers.12": 0,
    "model.layers.13": 0,
    "model.layers.14": 0,
    "model.layers.15": 0,
    "model.layers.16": 0,
    "model.layers.17": 0,
    "model.layers.18": 0,
    "model.layers.19": 0,
    "model.layers.20": 0,
    "model.layers.21": 0,
    "model.layers.22": 0,
    "model.layers.23": 0,
    "model.layers.24.self_attn": 0,
    "model.layers.24.mlp.gate_proj": 0,
    "model.layers.24.mlp.up_proj": "cpu",
    "model.layers.24.mlp.down_proj": "cpu",
    "model.layers.24.mlp.act_fn": "cpu",
    "model.layers.24.input_layernorm": "cpu",
    "model.layers.24.post_attention_layernorm": "cpu",
    "model.layers.25": "cpu",
    "model.layers.26"

In [22]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load peft config for pre-trained checkpoint
peft_model_id = "saved-peft-model-cause"
config = PeftConfig.from_pretrained(peft_model_id)

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_compute_dtype=compute_dtype,
    llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             return_dict=True,
                                             device_map={"": Accelerator().local_process_index},
                                             offload_folder="/tmp/.offload",
                                             quantization_config= bnb_config,
                                             pretraining_tp=1,
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Peft model loaded")

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:17<00:00,  4.33s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Peft model loaded


In [23]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(

In [24]:
print(dataset["instruction"][0])

<s>[INST]
"conversation": [
{
"utterance_ID": 1
"text": Please do not do that again . It is a horrible sound .
"speaker": Chandler
"emotion": disgust
}
{
"utterance_ID": 2
"text": Uh , it is Paul .
"speaker": Paul
"emotion": neutral
}
{
"utterance_ID": 3
"text": Buzz him in .
"speaker": Monica
"emotion": neutral
}
{
"utterance_ID": 4
"text": Who is Paul ?
"speaker": Joey
"emotion": surprise
}
{
"utterance_ID": 5
"text": Paul , the wine guy , Paul ?
"speaker": Ross
"emotion": neutral
}
{
"utterance_ID": 6
"text": Maybe .
"speaker": Monica
"emotion": neutral
}
{
"utterance_ID": 7
"text": Wait a minute . Your " not a real date " is with Paul , the wine guy ?
"speaker": Joey
"emotion": surprise
}
{
"utterance_ID": 8
"text": He finally asked you out ?
"speaker": Ross
"emotion": surprise
}
{
"utterance_ID": 9
"text": Yes .
"speaker": Monica
"emotion": joy
}
{
"utterance_ID": 10
"text": Ooh . This is a " Dear Diary " moment .
"speaker": Chandler
"emotion": surprise
}
{
"utterance_ID": 11
"tex

In [25]:
# Define folder for saving the output
if with_labels:
    eval_results_folder = "./results_train_cause"
    if not os.path.exists(eval_results_folder):
        os.makedirs(eval_results_folder)
else:
    eval_results_folder = "./results_test_cause"
    if not os.path.exists(eval_results_folder):
        os.makedirs(eval_results_folder)

In [26]:
n = len(dataset["instruction"])
print(n)

2274


In [27]:
from torch.utils.data import Dataset, DataLoader

In [28]:
# Dataloader for batched inference
class InferenceDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset["instruction"])

    def __getitem__(self, idx):
        instruction = self.dataset["instruction"][idx]

        return instruction

In [29]:
# Get full output for entire dataset, batched inference
def get_full_output_batched(dataset, batch_size=1):
    init_time = time.time()
    inference_dataset = InferenceDataset(dataset, tokenizer)

    data_loader = DataLoader(inference_dataset, batch_size=batch_size, shuffle=False)

    outputs_list = []

    model.eval()
    i = 0
    with torch.no_grad():
        for batch_item in data_loader:
            start_time = time.time()
            instructions = batch_item
            print("------------------------------------")
            print(f"Iteration {i}")
            print("------------------------------------")
            i += 1
            tokenized_instruction = tokenizer(instructions, return_tensors="pt", padding=True, truncation=True, max_length=3000)
            input_ids = tokenized_instruction["input_ids"].cuda()

            outputs = model.generate(input_ids=input_ids, max_new_tokens=25) # max_new_tokens is to limit the num of tokens added to orig input
            # since we only want to add utterance_ids :: [1,2,3,4,5] we may set it to 30

            decoded_outputs = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)
            end_time = time.time()
            print("Inference time for one sample: {:.2f} seconds".format((end_time - start_time)/batch_size))
            outputs_list.extend(decoded_outputs)
    final_time = time.time()
    print("Total time for {} samples using batched inferencing = {} seconds.".format(n, final_time - init_time))
    return outputs_list

In [30]:
preds_list = []
true_list = []
full_output = []

In [31]:
init_time = time.time()
full_output = get_full_output_batched(dataset,1)
final_time = time.time()

print("Total time for {} samples = {} seconds.".format(n, final_time - init_time))

D:\MiniConda\envs\pythonProject6\lib\site-packages\transformers\generation\utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


------------------------------------
Iteration 0
------------------------------------


D:\MiniConda\envs\pythonProject6\lib\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Inference time for one sample: 4.84 seconds
------------------------------------
Iteration 1
------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Inference time for one sample: 4.05 seconds
------------------------------------
Iteration 2271
------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Inference time for one sample: 4.11 seconds
------------------------------------
Iteration 2272
------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Inference time for one sample: 4.04 seconds
------------------------------------
Iteration 2273
------------------------------------
Inference time for one sample: 4.03 seconds
Total time for 2274 samples using batched inferencing = 8758.629425287247 seconds.
Total time for 2274 samples = 8758.629425287247 seconds.


In [32]:
def extract_cause_list(output):
    lst = output.split('::')
    if len(lst) >= 3:
        output = lst[2]
    else:
        output = lst[0]
    lst = output.split('[')
    if len(lst) >= 2:
        output = lst[1]
    else:
        output = lst[0]
    output = output.strip()
    lst = output.split(']')
    output = lst[0].strip()
    output = output.split(',')
    final = []
    for string in output:
        final.append(int(string))
    return final

In [33]:
pred_causes_map = {}
def evaluate_with_label_batched(dataset):
    # extract the list from text, !without shuffle
    for i in range(len(dataset["instruction"])):
        print("------------------------------------")
        print(f"Iteration {i}")
        print("------------------------------------")
        output = full_output[i] # of the form: utterance_ids :: [1,2] or []
        # extract
        output = extract_cause_list(output.strip())
        print("Predicted: ", output)
        preds_list.append(output)
        # baseline
        baseline_output = dataset["causes_list"][i]
        print("Baseline: ", baseline_output)
        true_list.append(baseline_output)

        # Update map
        conv_id = dataset["conversation_id"][i]
        utt_id = dataset["utterance_id"][i]
        if conv_id not in pred_causes_map.keys():
            pred_causes_map[conv_id] = {}
        pred_causes_map[conv_id][utt_id] = output

In [34]:
def evaluate_without_label_batched(dataset):

    # extract the label from text, !without shuffle
    for i in range(len(dataset["instruction"])):
        print("------------------------------------")
        print(f"Iteration {i}")
        print("------------------------------------")
        output = full_output[i] # of the form: utterance_ids :: [1,2] or []
        # extract
        output = extract_cause_list(output.strip())
        print("Predicted: ", output)
        preds_list.append(output)

        # Update map
        conv_id = dataset["conversation_id"][i]
        utt_id = dataset["utterance_id"][i]
        if conv_id not in pred_causes_map.keys():
            pred_causes_map[conv_id] = {}
        pred_causes_map[conv_id][utt_id] = output

In [35]:
if with_labels:
    evaluate_with_label_batched(dataset)
else:
    evaluate_without_label_batched(dataset)

------------------------------------
Iteration 0
------------------------------------
Predicted:  [1]
------------------------------------
Iteration 1
------------------------------------
Predicted:  [4]
------------------------------------
Iteration 2
------------------------------------
Predicted:  [7]
------------------------------------
Iteration 3
------------------------------------
Predicted:  [8]
------------------------------------
Iteration 4
------------------------------------
Predicted:  [9]
------------------------------------
Iteration 5
------------------------------------
Predicted:  [10]
------------------------------------
Iteration 6
------------------------------------
Predicted:  [10]
------------------------------------
Iteration 7
------------------------------------
Predicted:  [15]
------------------------------------
Iteration 8
------------------------------------
Predicted:  [6]
------------------------------------
Iteration 9
------------------------------

In [36]:
# Save results
with open(eval_results_folder + "/pred_text.json", 'w', encoding="utf-8") as f:
    for string in full_output:
        f.write(f"{string}\n[/NEW]\n")
with open(eval_results_folder + "/pred_list.json", 'w', encoding="utf-8") as f:
    for i in range(len(preds_list)):
        f.write(f'{preds_list[i]}\n')
if with_labels:
    with open(eval_results_folder + "/true_list.json", 'w', encoding="utf-8") as f:
        for i in range(len(true_list)):
            f.write(f'{true_list[i]}\n')
# Save dict
with open(eval_results_folder + "/pred_causes_map.json", 'w', encoding="utf-8") as f:
    json.dump(pred_causes_map, f)

In [37]:
with open("./results_test/emotion_labelled_data.json", 'r') as f:
    data = json.load(f)
for conv in data:
    conv_id = conv["conversation_ID"]
    conv["emotion-cause_pairs"] = []
    if conv_id not in pred_causes_map.keys():
        continue
    for utt_id, lst in pred_causes_map[conv_id].items():
        ff = str(utt_id)+"_"+conv["conversation"][utt_id - 1]["emotion"]
        for cause_id in lst:
            ss = str(cause_id)
            conv["emotion-cause_pairs"].append([ff, ss])
            print("Added pair: {}".format([ff, ss]))

Added pair: ['1_disgust', '1']
Added pair: ['4_surprise', '4']
Added pair: ['7_surprise', '7']
Added pair: ['8_surprise', '8']
Added pair: ['9_joy', '9']
Added pair: ['10_surprise', '10']
Added pair: ['11_fear', '10']
Added pair: ['15_surprise', '15']
Added pair: ['6_surprise', '6']
Added pair: ['3_surprise', '3']
Added pair: ['5_surprise', '5']
Added pair: ['6_sadness', '6']
Added pair: ['1_sadness', '1']
Added pair: ['2_surprise', '1']
Added pair: ['3_sadness', '3']
Added pair: ['7_sadness', '7']
Added pair: ['5_fear', '5']
Added pair: ['1_sadness', '1']
Added pair: ['3_sadness', '3']
Added pair: ['4_sadness', '4']
Added pair: ['5_sadness', '5']
Added pair: ['7_sadness', '7']
Added pair: ['8_sadness', '8']
Added pair: ['5_surprise', '5']
Added pair: ['7_sadness', '7']
Added pair: ['8_sadness', '8']
Added pair: ['10_surprise', '10']
Added pair: ['11_sadness', '10']
Added pair: ['14_sadness', '14']
Added pair: ['17_surprise', '17']
Added pair: ['22_joy', '21']
Added pair: ['24_joy', '2

In [38]:
with open(eval_results_folder + "/Subtask_2_pred.json", 'w', encoding="utf-8") as f:
    json.dump(data, f)

In [40]:
# If you want to download
# from google.colab import files
# import os

# dir_to_zip = eval_results_folder 
# output_filename = 'cause_results.zip' 
# delete_dir_after_download = "No"

# os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

# if delete_dir_after_download == "Yes":
#     os.system( "rm -r {}".format( dir_to_zip ) )

# files.download( output_filename )